In [ ]:
import geopandas as gpd
import numpy as np
import pandas as pd

In [ ]:
paths = {
    'censo_mgn': '../data/csv/CNPV2018_MGN_A2_11.csv',
    'censo_per': '../data/csv/CNPV2018_5PER_A2_11.csv',
    'cicloparqueaderos': '../data/shp/CICLOPARQUEADEROS.shp',
    'manzanas': '../data/shp/MANZANA.shp',
    'manzanas_mgn': '../data/shp/MGN_URB_MANZANA.shp',
    'parqueaderos': '../data/shp/PARQUEADEROS.shp',
    'viajes': '../data/csv/ViajesEODH_2019.csv',
    'zat': '../data/shp/ZAT.shp'
}

In [ ]:
df_censo_mgn = pd.read_csv(paths['censo_mgn'], index_col='COD_ENCUESTAS', dtype=str)
df_censo_per = pd.read_csv(paths['censo_per'], index_col='COD_ENCUESTAS', dtype=str)
df_viajes = pd.read_csv(paths['viajes'])
gdf_cicloparqueaderos = gpd.read_file(paths['cicloparqueaderos'])
gdf_manzanas = gpd.read_file(paths['manzanas'])
gdf_manzanas_mgn = gpd.read_file(paths['manzanas_mgn'])
gdf_parqueaderos = gpd.read_file(paths['parqueaderos'])
gdf_zat = gpd.read_file(paths['zat'], encoding='utf-8')

### Resumir información demográfica a nivel de encuesta

In [ ]:
dict_edad = {
    '1': np.nan,  # menores de 5 años no reportan viajes
    '2': '5-19',
    '3': '5-19',
    '4': '5-19',
    '5': '20-24',
    '6': '25-59',
    '7': '25-59',
    '8': '25-59',
    '9': '25-59',
    '10': '25-59',
    '11': '25-59',
    '12': '25-59',
    '13': '60+',
    '14': '60+',
    '15': '60+',
    '16': '60+',
    '17': '60+',
    '18': '60+',
    '19': '60+',
    '20': '60+',
    '21': '60+'
}

dict_educacion = {
    '1': 'MEDIA',
    '2': 'MEDIA',
    '3': 'MEDIA',
    '4': 'MEDIA',
    '5': 'SUPERIOR',
    '6': 'SUPERIOR',
    '7': 'SUPERIOR',
    '8': 'SUPERIOR',
    '9': 'POSGRADO',
    '10': 'NINGUNA',
    '99': np.nan
}

dict_sexo = {
    '1': 'HOMBRE', 
    '2': 'MUJER'
}

In [ ]:
def counts_per_group(df: pd.DataFrame, groupby_column: str, count_column: str, 
                     fillna: bool = True, droplevel: bool = True) -> pd.DataFrame:
    """
    Cuenta el número de registros para cada subgrupo en un DataFrame y almacena los
    conteos como una nueva columna para cada subgrupo.
    
    Parameters
    ----------
    df:              DataFrame original
    groupby_column:  nombre de la columna para crear los grupos
    count_column:    nombre de la columna para crear los subgrupos y realizar el conteo
    fillna:          reemplazar o no los NaNs con 0's
    droplevel:       eliminar o no el índice multijerárquico para las columnas
    
    Return
    ------
        DataFrame con los conteos para cada subgrupo como una columna
    
    """
    count = df.groupby([groupby_column, count_column])[count_column].count()
    count = count.rename('COUNT').reset_index(count_column).pivot(columns=count_column)
    
    if fillna:
        count = count.fillna(0)

    if droplevel:
        count.columns = count.columns.droplevel()
    
    return count

In [ ]:
columns = {'P_EDADR': dict_edad, 'P_NIVEL_ANOSR': dict_educacion, 'P_SEXO': dict_sexo}
for count_column, value_map in columns.items():
    df_censo_per[count_column] = df_censo_per[count_column].map(value_map)
    count = counts_per_group(df_censo_per, 'COD_ENCUESTAS', count_column)
    df_censo_mgn = df_censo_mgn.join(count)

### Resumir información demográfica a nivel de manzana censal y cruzar con capa de manzana censal

In [ ]:
df_censo_mgn['MANZ_CCNCT'] = df_censo_mgn['U_SECT_URB'] + df_censo_mgn['U_SECC_URB'] + df_censo_mgn['U_MZA']
df_censo_mgn = df_censo_mgn.set_index('MANZ_CCNCT')
df_censo_mgn = df_censo_mgn.drop(['U_SECT_URB', 'U_SECC_URB', 'U_MZA'], axis=1)

In [ ]:
df_censo_mgn = df_censo_mgn.groupby('MANZ_CCNCT').agg(sum)

In [ ]:
gdf_manzanas_mgn = gdf_manzanas_mgn.set_index('MANZ_CCNCT')
gdf_manzanas_mgn = gdf_manzanas_mgn.join(df_censo_mgn).dropna()

### Crear matriz origen destino con conteos por ZAT

In [ ]:
# fitrar ZATs para Bogotá
gdf_zat = gdf_zat.query('NOMMun == "Bogotá"')

In [ ]:
# filtrar viajes bicicleta
df_viajes = df_viajes.query('modo_principal == "Bicicleta"')

In [ ]:
# TODO ajustar viajes por f_exp

In [ ]:
df = pd.DataFrame(index=sorted(gdf_zat['ZAT'].unique()))

In [ ]:
df = df.join(df_viajes['zat_origen'].value_counts().rename('N_ORIGEN'))
df = df.join(df_viajes['zat_destino'].value_counts().rename('N_DESTINO'))

In [ ]:
# eliminar ZATs que no tienen viajes como origen o como destino
df = df.dropna()

### Cruzar capa de ZAT con capa de manzanas censales y resumir información demográfica a nivel de ZAT

In [ ]:
gdf_manzanas_mgn['AREA'] = gdf_manzanas_mgn.geometry.area
gdf_intersection = gpd.overlay(gdf_zat, gdf_manzanas_mgn.reset_index(), how='intersection')

In [ ]:
# La capa de ZAT presenta un gran número de errores topológicos y sus límites no coinciden
# exactamente con los límites de las manzanas censales en varios casos. Para evitar asignar
# información demográfica de una manzana censal a múltiples ZAT, se mantiene únicamente el 
# polígono de la intersección con mayor área para cada manzana censal.
gdf_intersection = gdf_intersection.sort_values(['MANZ_CCNCT', 'AREA'], ascending=False)
gdf_intersecttion = gdf_intersection.drop_duplicates('MANZ_CCNCT')

In [ ]:
gdf_intersection = gdf_intersection.drop(['MUNCod', 'NOMMun', 'UTAM', 'MANZ_CCNCT'], axis=1)
agg = gdf_intersection.groupby('ZAT').agg(sum)

In [ ]:
df = df.join(agg.drop('AREA', axis=1))

### Cruzar capa de ZAT con capa de manzanas y resumir información de estrato y uso del suelo a nivel de ZAT

In [ ]:
gdf_intersection = gpd.overlay(gdf_zat, gdf_manzanas, how='intersection')

In [ ]:
# obtener estrato con mayor area total dentro de cada ZAT
agg = gdf_intersection.groupby(['ZAT','ESTRATO'])['geometry'].agg(lambda x: x.area.sum())
top = agg.groupby(level=0, group_keys=False).nlargest(1)
df = df.join(top.reset_index('ESTRATO').drop('geometry', axis=1))

In [ ]:
# obtener uso con mayor area total dentro de cada ZAT
agg = gdf_intersection.groupby(['ZAT','USO'])['geometry'].agg(lambda x: x.area.sum())
top = agg.groupby(level=0, group_keys=False).nlargest(1)
df = df.join(top.reset_index('USO').drop('geometry', axis=1))

### Agregar número de parqueaderos y cupos de cicloparqueaderos por ZAT

In [ ]:
gdf_intersection = gpd.sjoin(gdf_zat, gdf_cicloparqueaderos, op='intersects')
count = gdf_intersection.groupby('ZAT')['CUPOS'].sum()
df = df.join(count)
df['CUPOS'] = df['CUPOS'].fillna(0)

In [ ]:
gdf_intersection = gpd.sjoin(gdf_zat, gdf_parqueaderos, op='intersects')
count = gdf_intersection['ZAT'].value_counts().rename('PARQUEADEROS')
df = df.join(count)
df['PARQUEADEROS'] = df['PARQUEADEROS'].fillna(0)